# TQuant Lab 交易策略開發實作-以月營收成長率為選股指標

本範例運用TQuant Lab為研究工具，以月營收成長率作為選股指標建立交易策略，並回測分析其績效表現。交易策略建立流程涵蓋以下 7 個步驟，以下依此分為 7 個章節說明。

1. 匯入套件 (Import Packages)

2. 獲取歷史資料 (Get History Data) 
3. 將交易資料綁入zipline回測架構中 (Bundle)
4. 因子研究 (Factor Research)
5. 建構策略 (Strategy Developement)
6. 回測 (Backtest)
7. 策略績效分析 (Performance Analysis)

## 1. 匯入套件 (Import Packages)

本節示範匯入TQuant Lab 的3個主要套件，分別說明相關功能、安裝方法與說明網頁。另外也匯入python常用的套件。
- **`TejToolAPI`: 收集資料、資料清洗<br>**
     - pip install tej-tool-api
        - https://pypi.org/project/tej-tool-api/
        - 設定api_key -> 參加試用 or 線上購買
        
- **`alphalens:` 因子研究**


- **`zipline:` 建構指標、回測**
    - https://pypi.org/project/zipline-tej/

- **`pyfolio:` 策略績效分析**
- **`pandas, numpy, matplotlib, seaborn: 其他常用套件`**

In [2]:
import os
os.environ['TEJAPI_KEY'] = "EOCprUG3A6FZFta9iUPXWYu8JQgjmY" 
os.environ['TEJAPI_BASE'] = "https://api.tej.com.tw/"
import TejToolAPI
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alphalens
import seaborn as sns
from scipy import stats
from zipline.pipeline import Pipeline
from zipline.pipeline.data import TWEquityPricing
from zipline.pipeline.factors import CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
from logbook import Logger, StderrHandler, INFO

In [3]:
log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

## 2. 獲取歷史資料 Get History Data

本節示範收集建立交易策略需要的相關資料，並展示儲存和取出方法，並分3小節依序說明。

### 2.1 TejToolAPI.get_history_data
使用 `TejToolAPI.get_history_data` 一鍵整併市場、財務、月營收和籌碼數據。

參數使用:
- tickers: 上市櫃全樣本(含下市櫃) <br>
- columns:  <br>
    - 屬性資料: 產業別
    - 市場資料: 開盤價, 收盤價, roi<br>
    - 籌碼資料: 外資買賣超金額_元<br>
    - 月營收: 月營收成長率_YoY<br>
    - 財務資料: 營業毛利率, 營業利益率, 稅後淨利率,  業外收支率, 營收成長率,
 營業毛利成長率, 營業利益成長率, 稅後淨利成長率, 淨值成長率, eps<br>
<br>
- period:
    - start: 2020-01-01
    - end  : 2023-11-08


In [4]:
from zipline.sources.TEJ_Api_Data import get_universe

start = '2020-01-01'
end = '2023-11-08'

start_dt = pd.Timestamp(start, tz='utc')
end_dt = pd.Timestamp(end, tz='utc')


# 获取在 2020 年期间台湾证券交易所和柜台买卖市场的普通股票池

pool = get_universe(start, end, mkt = ['TWSE', 'OTC'], stktp_e = 'Common Stock')
pool[0:10]

[2024-06-05 02:40:41.513909]: INFO: get_universe_TW: Filters：{'mkt': ['TWSE', 'OTC'], 'stktp_e': 'Common Stock'}


Currently used TEJ API key call quota 16/1000 (1.6%)
Currently used TEJ API key data quota 1544956/10000000 (15.45%)


['1101',
 '1102',
 '1103',
 '1104',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1240',
 '1259',
 '1264',
 '1268',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1333',
 '1336',
 '1339',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1454',
 '1455',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1470',
 '1471',
 '1472',
 '1473',
 '1474',
 '1475']

In [5]:
columns = ['Industry_Eng','開盤價','收盤價', 'roi', 'YoY_Monthly_Sales', 'eps', '外資買賣超金額_元','營業毛利率', '營業利益率', '稅後淨利率', '業外收支率', '營收成長率', '營業毛利成長率', '營業利益成長率', '稅後淨利成長率', '淨值成長率','Inventories', 'mktcap']
data = TejToolAPI.get_history_data(ticker=pool, columns=columns, transfer_to_chinese=False, start = start, end = end)
data = data.sort_values(['coid','mdate'])
data

Currently used TEJ API key call quota 263/1000 (26.3%)
Currently used TEJ API key data quota 8377617/10000000 (83.78%)


,coid,mdate,Industry_Eng,Open,ROI,Market_Cap_Dollars,Close,Qfii_Diff_Amt,YoY_Monthly_Sales,Basic_Earnings_Per_Share_A,...,Operating_Income_Rate_percent_TTM,Operating_Income_Growth_Rate_A,Operating_Income_Growth_Rate_Q,Operating_Income_Growth_Rate_TTM,Net_Income_Rate_percent_A,Net_Income_Rate_percent_Q,Net_Income_Rate_percent_TTM,Net_Non_operating_Income_Ratio_A,Net_Non_operating_Income_Ratio_Q,Net_Non_operating_Income_Ratio_TTM
0,1101,2021-01-04,M1100 Cement,43.20,0.0000,2.510030e+11,43.20,114998.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1101,2021-01-05,M1100 Cement,43.25,-0.2315,2.504220e+11,43.10,-63055.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1101,2021-01-06,M1100 Cement,43.10,-0.3480,2.495504e+11,42.95,-53387.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1101,2021-01-07,M1100 Cement,42.95,-0.2328,2.489694e+11,42.85,-205252.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1101,2021-01-08,M1100 Cement,42.90,0.2334,2.495504e+11,42.95,178200.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144345,9962,2023-11-02,OTC20 OTC Steel,17.15,0.5831,1.556299e+09,17.25,69.0,7.02,0.81,...,6.19,-26.22,-54.0,-7.26,4.25,3.72,5.24,0.21,0.49,0.39
1144346,9962,2023-11-03,OTC20 OTC Steel,17.35,0.5797,1.565322e+09,17.35,243.0,7.02,0.81,...,6.19,-26.22,-54.0,-7.26,4.25,3.72,5.24,0.21,0.49,0.39
1144347,9962,2023-11-06,OTC20 OTC Steel,17.40,1.4409,1.587877e+09,17.60,686.0,7.02,0.81,...,6.19,-26.22,-54.0,-7.26,4.25,3.72,5.24,0.21,0.49,0.39
1144348,9962,2023-11-07,OTC20 OTC Steel,17.60,0.5682,1.596899e+09,17.70,372.0,-23.55,0.81,...,6.19,-26.22,-54.0,-7.26,4.25,3.72,5.24,0.21,0.49,0.39


In [15]:
import pandas as pd

# 获取历史数据，包含 'pbr' 列
columns = ['coid', 'mdate', 'PBR_TWSE']
try:
    data = TejToolAPI.get_history_data(ticker=pool, columns=columns, transfer_to_chinese=False, start=start, end=end)
    
    # 对数据按公司 ID 和日期排序
    data = data.sort_values(['coid', 'mdate'])

    # 打印列名以确认实际存在的列
    print(data.columns)

    # 提取所需的股票代码和股价净值比两列
    if 'pbr' in data.columns:
        filtered_data = data[['coid', 'PBR_TWSE']]
        # 移除重复值（如果需要）
        filtered_data = filtered_data.drop_duplicates()
        # 显示结果数据集
        print(filtered_data)
    else:
        print("Column 'pbr' not found in the dataset")
except Exception as e:
    print(f"Error fetching data: {e}")



Currently used TEJ API key call quota 193/1000 (19.3%)
Currently used TEJ API key data quota 4619022/10000000 (46.19%)
Index(['coid', 'mdate', 'PBR_TWSE'], dtype='object')
Column 'pbr' not found in the dataset


### 2.2 Store Data into DataBase
將資料存進資料庫

In [4]:
import sqlite3
# 創建或連接到SQLite數據庫
conn = sqlite3.connect('your_database.db')
# 假設你的DataFrame名稱為 your_table_name，並希望將其寫入名為'table_name'的表
table_name = '選股'
data['mdate'] = data['mdate'].astype('datetime64[ns]')
data.to_sql(table_name, conn, if_exists='replace', index=False)

conn.close()

NameError: name 'data' is not defined

### 2.3 Extract Data from Database
從資料庫取出資料

In [17]:
import sqlite3
conn = sqlite3.connect('your_database.db')
table_name = '選股'
script = f'''
select * from {table_name}
'''
new_data = pd.read_sql(script, conn)
new_data

,coid,mdate,PBR_TWSE
0,1101,2021-01-04 00:00:00,1.32
1,1101,2021-01-05 00:00:00,1.32
2,1101,2021-01-06 00:00:00,1.32
3,1101,2021-01-07 00:00:00,1.31
4,1101,2021-01-08 00:00:00,1.32
...,...,...,...
1144345,9962,2023-11-02 00:00:00,1.40
1144346,9962,2023-11-03 00:00:00,1.41
1144347,9962,2023-11-06 00:00:00,1.43
1144348,9962,2023-11-07 00:00:00,1.43


## 3. 將交易資料綁入zipline回測架構中 (Bundle)

zipline 為 TQuant Lab 的回測引擎，旨要實現模擬交易策略，回測特定歷史區間內，策略是否具有獲利能力。在策略模擬回測前，需要先準備研究樣本的價量資料，並綁入(Bundle)zipline，並在完成後取出資料檢視，過程涵蓋以下3個步驟。

1. 準備 zipline 回測要用的資料<br>
2. 使用 !zipline ingest -b tquant 綁定回測資料<br>
3. 使用 get_bundle 檢視回測資料<br>

### 3.1 準備資料

- `os.environ['ticker']`：上市櫃全樣本(含下市櫃) 的股票代碼與加權報酬指數代碼(IR0001)
- `os.environ['mdate']` ：設定需要價量資料的起訖日，為2020-01-01~2023-11-08

In [5]:
new_data

NameError: name 'new_data' is not defined

In [8]:
import sqlite3
import pandas as pd

new_data = pd.DataFrame({
    'coid': ['2331', '2318', '2304'],
    'mdate': ['2022-11-01', '2022-11-02', '2022-11-03'],
    'PBR_TWSE': [8, 8, 8]
})


# 转换日期列为 datetime 格式
new_data ['mdate'] = pd.to_datetime(new_data['mdate'])

# 连接到 SQLite 数据库（如果不存在则创建一个新的数据库）
conn = sqlite3.connect('stock_data.db')

# 将 new_data 追加到名为 'stock_prices' 的表中
new_data.to_sql('stock_prices', conn, if_exists='append', index=False)

# 关闭数据库连接
conn.close() 


In [10]:
import sqlite3
import pandas as pd

# 定义时间范围
start_date = '2023-11-08'

# 连接到 SQLite 数据库
conn = sqlite3.connect('stock_data.db')

# 查询数据
query = f"""
SELECT coid, mdate, PBR_TWSE
FROM stock_prices
WHERE mdate >= '{start_date}' AND PBR_TWSE < 0.6
"""

# 读取数据到 DataFrame
data = pd.read_sql(query, conn)

# 转换日期列为 datetime 格式
data['mdate'] = pd.to_datetime(data['mdate'])

# 对数据按公司 ID 和日期排序
data = data.sort_values(['coid', 'mdate'])

# 移除重复值（如果需要）
filtered_data = data[['coid', 'PBR_TWSE']].drop_duplicates()

# 显示结果数据集
print(filtered_data)

# 关闭数据库连接
conn.close()


    coid  PBR_TWSE
0   1103      0.52
1   1312      0.48
2   1314      0.49
3   1416      0.54
4   1451      0.55
5   1718      0.50
6   1906      0.46
7   2104      0.55
8   2107      0.59
9   2362      0.47
10  2506      0.45
11  2514      0.51
12  2603      0.54
13  2609      0.53
14  3252      0.54
15  3481      0.50
16  5512      0.49
17  8354      0.56
18  9944      0.58


In [42]:
import os

# 获取当前工作目录
current_dir = os.getcwd()

# 将 CSV 文件保存在当前工作目录下
filtered_data.to_csv(os.path.join(r"C:\Users\ADMIN\Downloads\tmp", 'filtered_data(1).csv'), index=True)


In [43]:
# 将 DataFrame 写入到 Excel 文件
filtered_data.to_excel(os.path.join(r"C:\Users\ADMIN\Downloads\tmp", 'filtered_data(1).xlsx'), index=True)


### 3.2 綁入資料 

### 3.3 檢視資料

運用`get_bundle`函數取出Bundle入zipline的資料，需要設定以下4個參數。 

 - `bundle_name  `: tquant
 - `calendar_name`: TEJ
 - `start_dt     `: 2020-01-01
 - `end_dt       `: 2023-11-08

## 4. 因子研究 (Factor Research)
使用 alphalens 進行因子分析
- Returns（報酬率分析）
- Information（資訊分析）
- Autocorrelation（自相關分析）

In [12]:

                                                                   )

SyntaxError: unmatched ')' (1729570161.py, line 1)

## 5. 建構策略 (Strategy Developement)

本章節將運用`zipline.pipeline`的相關函數來建立交易策略的買賣訊號。<br>
以下過程:<br> 
**(1)** 定義 `CustomDataset` 函數: 定義策略需要的基本面變數(月營收成長率、毛利成長率、營業利益成長率、淨利成長率、產業別)。<br>
 **(2)** 使用 `DataFrameLoader` 函數，將第二節中用`TejToolAPI`函數抓取的基本面資料寫入`CustomDataset`函數定義的變數。<br>
**(3)** 定義 `compute_signals` 函數來設定篩選條件排除不符合條件的股票，以及門檻條件過濾出買進與賣出的股票，並轉換成買賣訊號。<br> 
**(4)** 最後利用 `run_pipeline` 函數產出買賣訊號整合成DataFrame輸出。

### 5.1 Define CustomDataset
創建 CustomDataset 物件 (該物件需繼承 `zipline.pipeline.data.dataset.DataSet`)。CustomDataset 定義每個欄位的資料型態，EX: int, float, str。另外也需要定義資料的地區(如本範例使用台灣地區的交易日)。

- 欄位:<br>
    - 月營收成長率_YoY: YoY_Monthly_Sales<br>
    - 毛利成長率: Gross_Margin_Growth_Rate_A<br>
    - 營業利益成長率: Operating_Income_Rate_percent_A<br>
    - 淨利成長率: Net_Income_Rate_percent_A<br>
    - 產業別: Industry<br>
- domain (地區):<br>
    - 透過domain可以限制該Dataset只能使用在domain相同的Pipelines中。
    - domain中包含calendar及two-character country code兩種資訊，台灣的domain為TW_EQUITIES、country code為TW、calendar為TEJ或TEJ_XTAI。 (modified by TEJ Research Team)

### 5.2 Transform Data Throught DataFrameLoader
透過 `DataFrameLoader` 將基本面資料寫入`CustomDataset` 對應的欄位。

### 5.3 Create Signal
利用`CustomDataset`的欄位定義交易策略的篩選、過濾指標。<br>
以`毛利成長率>0`、`營業利益率>0`、`淨利成長率>0`、為篩選指標，排除不符合此條件的股票。使用`月營收成長率`作為排序指標，買進月營收排名前 30 的股票，以此作為買賣的交易訊號。

In [ ]:
f

### 5.4 Run Pipeline
利用`run_pipeline`函數，將上一小節使用的相關欄位、定義的篩選指標、買賣的交易訊號整併，並輸出成MultiIndex(level_0=date, level_1asset)的DataFrame輸出。

## 6. 回測 (Backtest)
zipline 核心功能
1. 定義 initialize 設定

2. 定義 rebalance 條件設定
3. 定義 record_vars 交易過程資訊
4. 定義 before_trading_start
5. 執行回測

### 6.1 Define initialize
屬性:
- context.n_longs: 長部位檔數
- context.n_shorts: 短部位檔數
- context.min_positions: 最小持倉數
- context.universe: 股票池(同前面定義的ticker)
- context.tradeday: 執行交易日(未定義則為每日交易)
- context.set_benchmark: 設定set_benchmark<br>

設定 benchmark:<br>
context.set_benchmark(symbol('IR0001')) -> IR0001 為台灣加權報酬指數

設定交易成本:<br>
set_commission(commission.PerDollar(cost=commission_cost))

設定滑價:<br>
set_slippage(slippage.FixedSlippage(spread=0.00))

### 6.2 Define Rebalance 定義**再平衡**條件

若當前時點`get_datetime().strftime('%Y-%m-%d')`為預先設定之交易日（`context.tradeday`），則進行再平衡。  


#### 下單方式：
##### 1. Cancel

```python
## Cancel
open_orders = get_open_orders()
for asset in open_orders:
    for i in open_orders[asset]:
        cancel_order(i)   
```
- 取得（`get_open_orders()`）並取消（`cancel_order(i)`）帳上所有未完全成交的訂單。

##### 2. Divest
```python
## Divest
for stock, trade in context.trades.items():
    if not trade:
        order_target(stock, 0)
    else:
        trades[trade].append(stock)
```
- `stock`與`trades`：
  - 來自於`context.trades`，而`context.trades`來自於pipeline中的`'longs'`欄位，並在`before_trading_start`階段產出。
  - `'longs'`欄位是布林值（如5.4節）。若為True，則代表該股票是本期預計要持有的標的。
  - 迴圈中的`stock`是標的名稱、`trades`為0或1。若為本期預計要持有的標的則`trades=1`，反之為0。
- 若`trades=0`，代表本期**不需要**持有該檔股票，所以透過`order_target(stock, 0)`，出清帳上所有持股。
- 若`trades=1`，代表本期**需持有**該檔股票，這邊透過`trades[trade].append(stock)`將所有要持有的股票，存入`trades`中（`trades`是一個`defaultdict(list)`）。
<br> 
<br>

##### 3. Long
```python
## Long Only
context.longs = len(trades[1])
for stock in trades[1]:
    order_target_percent(stock, 1 / context.longs * 0.8)
```
   - `trades[1]`：取出`trades=1`的股票利用迴圈方式搭配`order_target_percent`下單。
   - 權數為`1 / context.longs * 0.8`。其中，`context.longs`為本期預計要持有的標的數目；而0.8是緩衝值，避免動用到槓桿（可以設定0~1的值）。實質上這個加權方式就是**等權重**加權。 

### 6.3 Define Record variables
紀錄交易的中間資訊
- context.account.leverage: 帳戶的槓桿比率


- context.longs
- context.shorts

### 6.4 Define before_trading_start
- output: 產生 pipeline 運算後的 signals


- context.trades: 是一個pandas.Series，將本期預計要持有的標的標示為1，反之為0。

### 6.5 Run Algorithm
執行回測

## 7. 策略績效分析 (Performance Analysis)

運用`PyFolio`進行Performance Analysis 


### 7.1 Detail in Transaction
- returns
- positions
- transactions

### 7.2 Performance Visualization
- 敘述統計表
- 累計報酬率圖表
- 最大回檔
- 前10大持股
- rolling beta, volatility, sharpe ratio  